### Classes and modules

In [ ]:
#Lets have matplotlib "inline"
%matplotlib inline

#Import packages we need
import numpy as np
from netCDF4 import Dataset
import xarray as xr

#For plotting
import matplotlib.pyplot as plt

plt.rcParams["lines.color"] = "w"
plt.rcParams["text.color"] = "w"
plt.rcParams["axes.labelcolor"] = "w"
plt.rcParams["xtick.color"] = "w"
plt.rcParams["ytick.color"] = "w"

from importlib import reload

In [ ]:
casename = 'skagerak'

from gpuocean.utils import NetCDFInitialization
reload(NetCDFInitialization)
case_info = NetCDFInitialization.getCaseLocation(casename)
case_info.pop("name")

In [ ]:
#s_source_url = "C:\\Users\\florianb\\Downloads\\ocean_his.an.20190716.nc"
s_source_url = "https://thredds.met.no/thredds/dodsC/fou-hi/new_norkyst800m/his/ocean_his.an.20190716.nc"

In [ ]:
s_nc = Dataset(s_source_url)

In [ ]:
xs = np.arange(case_info["x0"],case_info["x1"])
ys = np.arange(case_info["y0"],case_info["y1"])
t  = 0

In [ ]:
s_lvls = s_nc["Cs_r"][:].data
w_lvls = s_nc["Cs_w"][:].data

In [ ]:
s_temps = s_nc["temp"][t,:,case_info["y0"]:case_info["y1"],case_info["x0"]:case_info["x1"]]
s_sals  = s_nc["salt"][t,:,case_info["y0"]:case_info["y1"],case_info["x0"]:case_info["x1"]]

In [ ]:
s_hs   = s_nc["h"][case_info["y0"]:case_info["y1"],case_info["x0"]:case_info["x1"]]
s_lats = s_nc["lat_rho"][case_info["y0"]:case_info["y1"],case_info["x0"]:case_info["x1"]]

In [ ]:
import seawater as sw

In [ ]:
s_depths = np.ma.array(np.multiply.outer(s_lvls,s_hs), mask=s_temps.mask.copy())
w_depths = np.ma.array(np.multiply.outer(w_lvls,s_hs), mask=np.array((len(w_lvls)*[s_temps[0].mask.copy()])))

In [ ]:
s_pressures = sw.eos80.pres(-s_depths,s_lats)

In [ ]:
s_pot_densities = sw.eos80.pden(s_sals,s_temps,s_pressures)

In [ ]:
thres = 1024

In [ ]:
mld_idx = np.abs(s_pot_densities - thres).argmin(axis=0)

In [ ]:
min_idx = np.argmin(w_depths < -3, axis=0)

In [ ]:
mld_idx = np.ma.minimum(mld_idx,min_idx)

In [ ]:
mld = np.take_along_axis(s_depths, mld_idx.reshape(1,250,500), axis=0)[0]
w_mld = np.take_along_axis(w_depths, mld_idx.reshape(1,250,500), axis=0)[0]

In [ ]:
plt.imshow(mld, origin="lower", vmin=-10)
plt.colorbar(label="m",shrink=.6)
plt.title("Mixed layer depth ("+str(thres)+"kg/m3)")

In [ ]:
from scipy.ndimage.filters import convolve, gaussian_filter
min_mld = 3
#mld = np.ma.array(convolve(np.ma.minimum(mld,-min_mld), [[1/9,1/9,1/9],[1/9,1/9,1/9],[1/9,1/9,1/9]]), mask=s_temps[0].mask.copy())
mld = np.ma.array(gaussian_filter(np.ma.minimum(mld,-min_mld), [1,1]), mask=s_temps[0].mask.copy())
w_mld = np.ma.array(gaussian_filter(np.ma.minimum(w_mld,-min_mld), [1,1]), mask=s_temps[0].mask.copy())

In [ ]:
plt.imshow(np.ma.array(s_hs, mask=s_temps[0].mask.copy()), origin="lower", cmap="ocean")
plt.scatter(np.repeat(225,len(ys)), np.arange(len(ys)), c="red", s=5.0)

In [ ]:
s_pot_densities_show = np.ma.array(np.zeros((len(ys),1000)))
for l in range(len(ys)):
    d_up = 0 
    for i in reversed(range(len(s_lvls))):
        d = round(-(s_hs[l,225]*s_lvls[i]))
        s_pot_densities_show[l,d_up:d] = s_pot_densities[i,l,225]
        d_up = d
s_pot_densities_show.mask = (s_pot_densities_show<1)

idxs_show = np.arange(len(s_pot_densities_show.T[0]))[s_pot_densities_show.T[0].mask == False]

In [ ]:
fig, axs = plt.subplots(2,1,figsize=(12,6))

im = axs[0].imshow(s_pot_densities_show.T[0:25,idxs_show], vmin=1020, vmax=1028, cmap="plasma",  aspect='auto')
axs[0].set_ylabel("Depth \n(zoom in)")
axs[0].set_title("Potential Densities")

axs[0].scatter(np.arange(len(idxs_show)),-mld[idxs_show,225], c="green", s=2.5)
axs[0].scatter(np.arange(len(idxs_show)),-w_mld[idxs_show,225], c="cyan", s=2.5)


im = axs[1].imshow(s_pot_densities_show.T[0:700,idxs_show], vmin=1020, vmax=1028, cmap="plasma",  aspect='auto')
axs[1].set_ylabel("Depth \n(full)")
plt.xlabel("Skagen <<<   >>> Arendal")

fig.subplots_adjust(right=0.8)
cbar_ax = fig.add_axes([0.85, 0.15, 0.02, 0.7])
fig.colorbar(im, cax=cbar_ax, label="kg/m3")
